In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

np.random.seed(1122)

## Bernoulli Coin Flips
Write a function that takes a list of 0s and 1s that represent a series of coin tosses, with 1s meaning heads and 0s meaning tails.
Have your function output the log likelihood of observing the list of results given they were generated using a fair coin.

Recall that the equation for the PMF of this type of random variable is $ f(x) = p^x(1-p)^{(1-x)} $
This tells you the probability of observing a value of x (which is 0 or 1). To find the joint probability of a set of coin flips, you can multiply the probability of each flip together.
We want the log-likelihood, and remember that logs turn products into sums.

In [ ]:
def bernoulli_LL(coin_tosses):
    # fill in function here
    
    return LL


num_flips = 10
coin_tosses = np.random.choice([0,1], size=num_flips)
print(bernoulli_LL(coin_tosses))

Modify your function (copy-paste it below) to also take a value between 0 and 1 as an argument. Your new function should treat this value as the probability of flipping heads and return the appropriate log-likelihood

In [ ]:
def bernoulli_LL(coin_tosses, p_heads):
    # fill in function here
    
    return LL

print(bernoulli_LL(coin_tosses, 0.25))

Write a function that takes a series of coin tosses as before, but returns the probability of flipping heads that is most likely by maximizing the log-likelihood. To find the maximum, use the 1st derivative.

In [ ]:
def fit_bernoulli(coin_tosses):
    # fill in function here

    return p

num_flips = 10
coin_tosses = np.random.choice([0,1], size=num_flips)
best_p = fit_bernoulli(coin_tosses)
print(best_p)

What value do you think you will approach as you increase num_flips? What about in the code below? See documentation for [numpy.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)

In [ ]:
num_flips = 10
# now lets tell random.choice to use a specific p_heads
coin_tosses = np.random.choice([0,1], size=num_flips, p=[0.58,0.42])
best_p = fit_bernoulli(coin_tosses)
print(best_p)

This is a direct demonstration of how increasing your statistical power can help you develop encoding models that are more likely to align with the true encoding scheme. Now let's see how random noise is effecting things. We will try gradually increasing num_flips and generating several sets of coin tosses at each value. Then we will plot the parameters of the bernoulli distributions we fit each time.

In [ ]:
np.random.seed(1122)
# let's try log-spaced flip quantities
num_flips = [10,100,1000,10000,100000]
fit_probabilities = np.ndarray((len(num_flips),10))
for nf_idx, nf in enumerate(num_flips):
    # let's get 10 samples each time
    for ct in np.arange(10):
        # each iterative call of random.choice will create a new set of coin tosses
        # this serves as our "noise" in this example
        coin_tosses = np.random.choice([0,1], size=nf)
        fit_probabilities[nf_idx,ct] = fit_bernoulli(coin_tosses)

# plot results
fit_means = np.mean(fit_probabilities, axis=1)
fit_stds = np.std(fit_probabilities, axis=1)
plt.plot(num_flips, fit_means, c='k')
plt.fill_between(num_flips, fit_means+fit_stds, fit_means-fit_stds, alpha=.5, color='b')
plt.gca().set_xscale('log')
plt.show()

Not only do we converge on the true value, but we are much more consistent.

## Gaussian Spiking Neuron
Now let's apply the same basic concept to modeling a spiking neuron as a linear-gaussian function of a binary stimulus. This is equivalent to using ordinary least squares regression, and is also a kind of GLM where our "nonlinearity" is linear and our noise distribution is a gaussian.

First let's load the data and see what it looks like. This data is from a single retinal ganglion cell being shown a binary gaussian white noise visual input (what was shown in the lecture).

In [ ]:
# make sure the RGC3.npz file is in your working directory, or provide the full path
data = np.load('RGC3.npz')['args']
stim = data[0] # the value of the stimulus at each timepoint
spikes_binned = data[1] # the number of spikes in each time bin. We have 1 bin per frame of stimulus
stim_times = data[2] # how long in absolute time has elapsed at each timepoint
dt_stim = stim_times[1] - stim_times[0] # the bin size in absolute time

# Let's visualize some of the raw data
fig, (ax1,ax2) = plt.subplots(2)
fig.set_size_inches(12,8)
iiplot = np.arange(120)
ttplot = iiplot*dt_stim
ax1.plot(ttplot, stim[iiplot])
ax1.set_title('raw stimulus')
ax1.set_ylabel('stim intensity')
ax1.set_xlim([ttplot[0],ttplot[-1]])

ax2.stem(ttplot,spikes_binned[iiplot])
ax2.set_xlim([ttplot[0], ttplot[-1]])
ax2.set_xlabel('time (s)')
ax2.set_title('binned spike counts')
ax2.set_ylabel('spike count')
ax2.set_xlim([ttplot[0],ttplot[-1]])
plt.tight_layout()
plt.show()

print('------------------------------------------------')
print(f'Our time bin size is about {dt_stim*1000:.1f} ms')

Now we need to make our design matrix so that we can conveniently use the dot product to keep N frames of the stimulus preceeding each spike. Basically we need a matrix in which each row contains N frames prior to each timepoint. This creates a problem for the first N timepoints, where there are some frames of stimulus preceeding them that don't exist. Since our stimulus varies between 0.48 and -0.48, we can just pretend there were stimulus frames set to 0, we can "pad" our stimulus with 0s.

Now create the design matrix where N is 25. It should look like the matrix that was in the slides.

In [ ]:
N=25 # use the last 10 frames
padded_stim = np.hstack((np.zeros((N-1)), stim)) # pad with 0s

# insert code here, you will need to do some looping

X = 

# The design matrix is huge, but we can just look at the beginning of it to make sure it looks right
# it should look similar to the one in the slides.
plt.clf()
plt.figure(figsize=[12,8])
plt.imshow(X[:50], aspect='auto', interpolation='none')
plt.xlabel('lags before spike time')
plt.ylabel('time bin of response')
plt.colorbar()
plt.show()

Now let's see what our encoding model finds under the assumptions that it is linear and distributed as a gaussian. Remember we can just use the least squares formula. This is the same thing as the spike-triggered average (STA).
$(X^TX)^{-1}X^TY$

We don't even need to include the stimulus covariance part, because our stimulus is gaussian white noise, but we will anyway.
Recall that in python, the dot product is @, and you can use [numpy.linalg.inv](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) to do the inverse or [numpy.linalg.solve](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html)

In [ ]:
# insert code here

thetas = 

# insert plotting code here


Now let's use our thetas to see what our encoding model says our spikes should look like. Normally, it would be good to make new predictions using stimulus you held out from training to see how well it generalizes, but for now we can just use our same design matrix. Take the dot product of your design matrix and your filter to generate the predicted spikes.

In [ ]:
# insert code here

### Plot real spike train and prediction
plt.clf()
plt.figure(figsize=[12,8])
plt.stem(ttplot,spikes_binned[iiplot], linefmt='b-', basefmt='k-', label="spike ct")
### plot the predictions here in a different color
plt.stem()
plt.ylabel('spike count'); plt.xlabel('time (s)')
plt.xlim([ttplot[0], ttplot[-1]])
plt.legend()
plt.show()

Did our model do a good job? Do you think the neuron is acting as linear-gaussian filters of the stimulus? Maybe the neuron has a set mean firing rate that is modulated by the stimulus, rather than purely firing based on the stimulus alone. We can incorporate this into our model by adding a constant offset to our design matrix, essentially an extra variable that is always set to 1.

In [ ]:
# insert code that adds a column of ones to your design matrix

# compute the new STA

# plot model prediction with and without the offset, along with the actual binned spikes

## Linear Non-linear Poisson Model

Now lets fit a Linear Non-linear Poisson (LNP / Poisson GLM) model to this neuron. Unlike in the linear case, we don't have a closed form solution to find the optimal thetas, so we will have to manually perform MLE. Fortunately the statsmodels module in python supports various kinds of [GLMs](https://www.statsmodels.org/stable/generated/statsmodels.genmod.generalized_linear_model.GLM.html#statsmodels.genmod.generalized_linear_model.GLM), including Poisson GLMs with an exponential nonlinearity, which is what we will use. Keep in mind that the vocabulary statsmodels documentation uses is the "link function" which is the *inverse* of the nonlinearity. So to run an exponential GLM you want a logarithm link function.

In [ ]:
# Package available for download from
# https://www.statsmodels.org/stable/install.html
import statsmodels.api as sm

# insert code here to fit poisson GLM, be sure to use your design matrix with the offset

# insert code here to obtain thetas

# insert code here to generate predicted spikes

# insert code here to plot LNP model predictions, along with linear gaussian model predictions, and raw spike rates


Which looks best? Which has the best $R^2$ with the raw data?

In [ ]:
# insert code here to compute R^2
